# **CMPT 726/419 A3 Q4: Neural Networks in PyTorch**

Do not edit any cells until told to do so—the ones directly below should not be changed so you can access the required data and model for this problem.

In [ ]:
from tqdm.notebook import tqdm
import csv

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR

from torchvision import transforms
from torchvision.datasets import CIFAR10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class XYDataset(Dataset):
    """A basic dataset where the underlying data is a list of (x,y) tuples. Data
    returned from the dataset should be a (transform(x), y) tuple.
    Args:
    source      -- a list of (x,y) data samples
    transform   -- a torchvision.transforms transform
    """
    def __init__(self, source, transform=transforms.ToTensor()):
        super(XYDataset, self).__init__()
        self.source = source
        self.transform = transform

    def __len__(self): return len(self.source)
    
    def __getitem__(self, idx):
        x,y = self.source[idx]
        return self.transform(x), y

def build_dataset():
    """Returns the subset of the CIFAR-10 dataset containing only horses and
    deer, with the labels for each class modified to zero and one respectively.
    """
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(32, scale=(0.75, 1.0)),
        transforms.ToTensor()])

    data = CIFAR10(root=".", train=True, download=True)
    data = [(x, (0 if y == 2 else 1)) for x,y in data if y in {2, 5}]
    return XYDataset(data, transform=transform)

In [ ]:
def generate_test_predictions(model):
    """Generates test predictions using [model]."""
    data_te = torch.load("cifar2_te.pt")
    loader = DataLoader(data_te, batch_size=128, num_workers=6, shuffle=False)
    preds = []
    with torch.no_grad():
        for x,_ in loader:
            fx = model(x.to(device))
            preds += (fx > .5).float().view(-1).cpu().tolist()

    with open("test_predictions.csv", "w+") as f:
        writer = csv.writer(f)
        writer.writerow(["Id", "Category"])
        for idx,p in enumerate(preds):
            writer.writerow([str(idx), str(int(p))])
    tqdm.write("Wrote model predictions to test_predictions.csv")

In [ ]:
class NN(nn.Module):
    """A simple ConvNet for binary classification."""
    def __init__(self):
        super(NN, self).__init__()
        self.c1 = nn.Conv2d(3, 32, kernel_size=3)
        self.c2 = nn.Conv2d(32, 32, kernel_size=3)
        self.fc = nn.Linear(25088, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        bs = len(x)
        fx = self.c1(x)
        fx = self.relu(fx)
        fx = self.c2(fx)
        fx = self.relu(fx)
        fx = fx.view(bs, -1)
        fx = self.fc(fx)
        fx = self.sigmoid(fx)
        return fx.view(-1)


### ----- EDIT NO CODE ABOVE THIS CELL -----
### ----- EDIT CODE BENEATH THIS CELL -----





In [ ]:
def validate(model, loader):
    """Returns a (acc_val, loss_val) tuple, where [acc_val] and [loss_val] are
    respectively the validation accuracy and loss of [model] on the data in
    [loader].

    Args:
    model   -- a model, already moved onto the GPU
    loader  -- a DataLoader over validation data
    """
    acc_val, loss_val = 0, 0
    loss_fn = nn.BCELoss(reduction="mean")
    ##### YOUR CODE STARTS HERE (redefine [acc_val] and [loss_val] somewhere) ##

    total = 0
    model.eval()
    with torch.no_grad():
      for data in loader:
        inputs, labels = data
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())
        loss_val += loss.item()
        predicted = (outputs > .5).float()
        total+= labels.size(0)
        acc_val += (predicted == labels).sum()

    acc_val = float(acc_val)/float(total) * 100

    ##### YOUR CODE ENDS HERE   ################################################
    return acc_val, loss_val

In [ ]:
def one_epoch(model, optimizer, loader):
    """Returns a (model, optimizer, avg_loss) tuple after training [model] on
    the data in [loader] for one epoch.

    Args:
    model       -- the neural network to train, already moved onto the GPU
    optimizer   -- the optimizer used to train [model]
    loader      -- a DataLoader with data to train [model] on

    Returns
    model       -- [model] after training for one epoch
    optimizer   -- the optimizer used to train [model]
    avg_loss    -- the average loss of [model] on each batch of [loader]
    """
    avg_loss = 0
    loss_fn = nn.BCELoss(reduction="mean")
    ##### YOUR CODE STARTS HERE (redefine [avg_loss] somewhere) ################

    for i, data in enumerate(loader, 0):
      inputs, labels = data
      outputs = model(inputs)
      optimizer.zero_grad()
      loss = loss_fn(outputs, labels.float())
      loss.backward()
      optimizer.step()
      avg_loss += loss.item()
    
    avg_loss = avg_loss/len(loader)

    ##### YOUR CODE ENDS HERE   ################################################
    return model, optimizer, avg_loss

Next, write a function to take a set of hyperparameters and return a `(model, acc_val)` tuple with the model having been trained with the hyperparameters and `acc_val` being the validation accuracy of the model after training.

Note the `**kwargs` in the function definition. This means that the function can take
any number of keyword arguments as input, and they will be accessible inside the function despite not being specified in the function definition, and they will be accessible within dictionary named `kwargs` inside the function.

_By passing in arguments this way, you can define whatever hyperparameters you want to use and pass them in!_

In [ ]:
def train_and_validate(data_tr, data_val, **hyperparameters):
    """Returns a (model, acc_val) tuple where [model] is a neural
    network of the NN class trained with [hyperparameters] on [data_tr] and 
    validated on [data_val], and [acc_val] is the validation accuracy of the
    model after training.

    Args:
    data_tr         -- Dataset of training data
    data_val        -- Dataset of validation data
    hyperparameters -- kwarg dictionary of hyperparameters
    """
    model, acc_val = NN(), 0
    ##### YOUR CODE STARTS HERE (redefine [model] and [acc_val] somewhere) #####

    learning_rate = hyperparameters.get('learning_rate')
    batch_size = hyperparameters.get('batch_size')
    num_epochs = hyperparameters.get('num_epochs')

    trainloader = DataLoader(data_tr, batch_size = batch_size, shuffle=True)
    validloader = DataLoader(data_val, batch_size = batch_size, shuffle=True)

    optimizer = Adam(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, learning_rate, num_epochs* len(data_tr))

    for epoch in range(num_epochs):
      model, optimizer, avg_loss = one_epoch(model, optimizer, trainloader)
      acc_val, loss_val = validate(model, validloader)
      scheduler.step()

    ##### YOUR CODE ENDS HERE   ################################################
    return model, acc_val

In [ ]:
################################################################################
# Do hyperparameter search using train_and_validate(). You should call
# generate_test_predictions() on the model you eventually compute as best to 
# get test predictions to submit to Kaggle. You should also split off some of
# the 
data = build_dataset()  # You should use this data for training and validation
best_model = NN()       # You should at some point name the model you want to
                        # generate test predictions with `best_model`
                        
##### YOUR CODE STARTS HERE ####################################################

trainset = data
testset = data

hyperparameters = {"learning_rate":0.001, "batch_size": 16, "num_epochs": 100}
best_model, acc_val = train_and_validate(trainset, testset, **hyperparameters)

##### YOUR CODE ENDS HERE   ####################################################

Files already downloaded and verified
0.6312441349983215 70.14
0.5676066842556 72.87
0.5437824920415878 74.0
0.5258740147590637 74.63
0.5135277182340622 75.44
0.5036107137441636 75.75
0.495849379324913 77.17
0.4871834241390228 77.35
0.4827282478094101 76.73
0.47520403723716736 77.95
0.47200482671260835 78.66
0.46749858434200287 78.77
0.46595187747478484 77.11
0.4561788381576538 79.69000000000001
0.4563820033073425 79.43
0.45225237033367155 79.22
0.4508535409450531 80.08999999999999
0.4456490130662918 78.36999999999999
0.44646070294380186 79.84
0.4429484364748001 79.47
0.4379809746265411 80.67999999999999
0.43423329815864564 80.60000000000001
0.43150466537475585 81.45
0.43538962285518645 81.11
0.43167245552539824 80.66
0.42726236174106597 81.35
0.41865457541942597 81.23
0.4227517702817917 81.43
0.42375799884796145 80.21000000000001
0.42017695264816285 81.92
0.4149860407114029 81.67999999999999
0.4206607741594315 82.1
0.41793514032363893 81.6
0.41507954103946687 81.6
0.4110004977941513 8

In [ ]:
generate_test_predictions(best_model)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Wrote model predictions to test_predictions.csv
